## Initialize a map object with geemap

Later in this tutorial, we're going to show you how to import other geospatial data files, such as the boundaries to countries or sub-national regions, to help you analyze nighttime lights.

For now, we're going to focus our scene on Nigeria, but we'll do that simply by centering our map on the capital city of Abuja, which we can find at approximately: latitude: 9.0 and longitude: 7.4. 

We'll set our map zoom factor to 6 to include the entire country in our view. 

The try and except blocks in the code below help with Caveats incase you catch an error. 

In [6]:
try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...") ## for google collab
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y   ## for your local directory
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [7]:
try:
        ee.Initialize()    ## This is done anytime you want to use the google earth engine API.
except Exception as e:
        ee.Authenticate()  ## To help google earth engine link your account
        ee.Initialize()

In [8]:
# set our initial map parameters for Abuja, Nigeria
center_lat = 9.0
center_lon = 7.4
zoomlevel=6

# initialize our map
myFirstMap = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)

# display our map
myFirstMap.addLayerControl()
myFirstMap

Map(center=[9.0, 7.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [9]:
## GRID3 population data
img_pop3 = ee.ImageCollection('users/henrique/GRID3_NGA_PopEst_v1_1_mean_float')
##Nigerian nightlights (1Y median)
nighttime = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
                  .filter(ee.Filter.date('2018–09–01', '2019–09–30')) \
                  .median() \
                  .select('avg_rad') \
                  .clipToCollection(myFirstMap)

## Night maps

In [10]:
start_date = '2019-01-01'
end_date =  '2019-12-31'


##VIIRS Nighttime Day/Night Band Composites Version 1

dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(start_date, end_date))

print(f"there are {dataset.size().getInfo()} images in this collection. One per month.")  ## Shows how many images in our collection.

start_date = '2020-01-01'
end_date =  '2020-12-31'

there are 12 images in this collection. One per month.


In [11]:
##VIIRS Nighttime Day/Night Band Composites Version 1

dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(start_date, end_date))
nighttime = dataset.select('avg_rad')
nighttimeVis = {min: 0.0, max: 100.0};
myFirstMap.setCenter(center_lat, center_lon, zoomlevel);
myFirstMap.addLayer(nighttime, nighttimeVis, '2020 Nighttime')

# display our night time map
myFirstMap.addLayerControl()
myFirstMap


Map(center=[7.4, 9.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

---

## Masked Maps

This section is for masked Maps. Masked Maps are Maps that negative and Zero values have been filtered out.

In [12]:
### Mask

start_date = '2020-01-01'
end_date =  '2020-12-31'

mySecondMap = geemap.Map()

##VIIRS Nighttime Day/Night Band Composites Version 1

dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(start_date, end_date))
nighttime = dataset.select('avg_rad').median() ##Median is used when masking
nighttimeVis = {min: 0.0, max: 60.0};
mySecondMap.setCenter(center_lat, center_lon, zoomlevel);
mySecondMap.addLayer(nighttime.mask(nighttime), nighttimeVis, '2020 Mask', opacity=0.75)  ##mask function called here.

mySecondMap

Map(center=[7.4, 9.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Adding BaseMap

In [13]:
### Adding BaseMap

start_date = '2020-01-01'
end_date =  '2020-12-31'

myThirdMap = geemap.Map()

myThirdMap.add_basemap("SATELLITE") ## BaseMap just means another layer of Map. 
## You can call different Maps as stored in the databases. You can read more on this. 
## Satellite BaseMap shows a typography of the Earth - kinda- from space.


dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(start_date, end_date))
nighttime = dataset.select('avg_rad').median()
nighttimeVis = {min: 0.0, max: 60.0};
myThirdMap.setCenter(center_lat, center_lon, zoomlevel);
myThirdMap.addLayer(nighttime.mask(nighttime), nighttimeVis, '2020 Mask', opacity=0.75)

myThirdMap

Map(center=[7.4, 9.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Split Maps

In [14]:
### Split Night MAps

start_date = '2013-01-01'
end_date =  '2013-12-31'

myLeftMap = geemap.Map()

dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(start_date, end_date))
nighttime = dataset.select('avg_rad')
nighttimeVis = {min: 0.0, max: 100.0}


myLeftMap.setCenter(center_lat, center_lon, zoomlevel)
myLeftMap.addLayer(nighttime, nighttimeVis, '2013 Nighttime')

In [15]:
start_datee = '2020-01-01'
end_datee =  '2020-12-31'

myRightMap = geemap.Map()

dataset_1 = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(start_datee, end_datee))
nighttime_1 = dataset_1.select('avg_rad')
nighttimeVis = {min: 0.0, max: 100.0}
myRightMap.setCenter(center_lat, center_lon, zoomlevel)
myRightMap.addLayer(nighttime_1, nighttimeVis, '2020 Nighttime')

In [16]:
# generate tile layers from the ee image objects, masking and changing opacity to 75%
tile_2013 = geemap.ee_tile_layer(nighttime, {}, 'Nighttime 2013', opacity=0.75)
tile_2020 = geemap.ee_tile_layer(nighttime_1, {}, 'Nighttime 2020', opacity=0.75)

In [17]:
# initial map object centered on Abuja. Our Split map will be called splitMap
splitMap = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)

# use .split_map function to create split panels
splitMap.split_map(left_layer=tile_2013, right_layer=tile_2020) ## split_map is the function here


splitMap.addLayerControl()
splitMap

Map(center=[9.0, 7.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Split Maps with masks

In [18]:
start_date = '2013-01-01'
end_date =  '2013-12-31'


dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(start_date, end_date))
nighttime = dataset.select('avg_rad').median()
nighttimeVis = {min: 0.0, max: 100.0}
myFirstMap.setCenter(center_lat, center_lon, zoomlevel)
myFirstMap.addLayer(nighttime, nighttimeVis, '2013 Nighttime')

In [19]:
start_datee = '2020-01-01'
end_datee =  '2020-12-31'


dataset_1 = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(start_datee, end_datee))
nighttime_1 = dataset_1.select('avg_rad').median()
nighttimeVis = {min: 0.0, max: 100.0}
myFirstMap.setCenter(center_lat, center_lon, zoomlevel)
myFirstMap.addLayer(nighttime_1, nighttimeVis, '2020 Nighttime')

In [20]:
# generate tile layers from the ee image objects, masking and changing opacity to 75%
tile_2013 = geemap.ee_tile_layer(nighttime.mask(nighttime), {}, 'Nighttime 2013', )
tile_2020 = geemap.ee_tile_layer(nighttime_1.mask(nighttime_1), {}, 'Nighttime 2020',)

In [21]:
# initial map object centered on Abuja
Map7 = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)

# use .split_map function to create split panels
Map7.split_map(left_layer=tile_2013, right_layer=tile_2020)


Map7.addLayerControl()
Map7


Map(center=[9.0, 7.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

---

In [23]:
## World Pop

Map= geemap.Map()

worldPop = ee.ImageCollection("WorldPop/GP/100m/pop") .filterDate('2018');
populationVis = {min: 0.0,max: 50.0, 'palette' : ['24126c', '1fff4f', 'd4ff50']};

Map.setCenter(9.0, 7.6, 6);
Map.addLayer(worldPop, populationVis, 'Population');

Map



Map(center=[7.6, 9.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…


## Exporting night time image

In this section, we will be analyzing our nightime image and try to get data from them.

In [ ]:
import os
import geemap
import ee

In [ ]:
try:
        ee.Initialize()    ## This is done anytime you want to use the google earth engine API.
except Exception as e:
        ee.Authenticate()  ## To help google earth engine link your account
        ee.Initialize()

In [ ]:
# set our initial map parameters for Abuja, Nigeria
center_lat = 9.0
center_lon = 7.4
zoomlevel=6

# initialize our map
myFirstMap = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)

# display our map
myFirstMap.addLayerControl()
myFirstMap

In [ ]:
## Displaying nighttime image

start_date = '2016-01-01'
end_date =  '2016-12-31'


## Creating night-time image from VIIRS dataset
dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(start_date, end_date))
nighttime = dataset.select('avg_rad')
nighttime_image = nighttime.median()
nighttimeVis = {min: 0.0, max: 100.0};
myFirstMap.setCenter(center_lat, center_lon, zoomlevel);
myFirstMap.addLayer(nighttime_image.mask(nighttime_image), nighttimeVis, '2016 Nighttime Mask')


# display our night time map
myFirstMap.addLayerControl()
myFirstMap

In [ ]:
## run night mask first and draw the box before running this or you can use the cordinates approach. This approach is 
## commented
feature = myFirstMap.draw_last_feature
feature

In [ ]:
print(feature)

In [ ]:
## run night mask first and draw the box before running this or you can use the cordinates approach. This approach is 
## commented
## feature = mySecondMap.draw_last_feature
## feature

# Draw any shapes on the map using the Drawing tools before executing this code block
feature = myFirstMap.draw_last_feature

if feature is None:
    geom = ee.Geometry.Polygon([[[7.809796,4.887219],
                                 [7.809796,5.160274],
                                 [8.402061,5.160274],
                                 [8.402061,4.887219],
                                 [7.809796,4.887219]]])
    feature = ee.Feature(geom, {})

## ROI is region of interest
roi = feature.geometry()

In [ ]:
## ascertain path to download image
path = "~\Desktop\PERSONAL\OMDENA\Google Earth Engine"
out_dir = os.path.join(os.path.expanduser(path), "Downloaded Maps")

filename = os.path.join(out_dir, "NigeriaMask2016.tif")

In [ ]:
###Export function

geemap.ee_export_image(nighttime_image, filename= filename, scale= 100, region = roi, file_per_band=False,)

---

## Night time image analysis

Using VIIRS as the raster input, examine its attributes, plot a histogram showing the distribution of the night time intesity (avg_rad), and showing a simple masking technique to remove non habitable areas.

The raster is already bounded from GEE.

1  Open Source Python Notebook using simple Rasterio library to examine raster

1.1  Extract a raster file and examine its attributes as well as examine a single band from that raster then show the image.¶
Check version & install libraries.

In [ ]:
# standard library
import os
from pprint import pprint

# common library
import numpy as np 
import pandas as  pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# GIS libraries
import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
import xarray as xr
import rioxarray as rxr

In [ ]:
## Put your downloaded geotiff image in the same folder.

data = "NigeriaMask2016.tif"

In [ ]:
# Open the file:
raster = rasterio.open(data)

In [ ]:
# from rasterio doc: attributes
print(f'Raster shape:\t\t {raster.shape}')
print(f'Raster band count:\t {raster.count}')
print(f'Raster data types:\t {raster.dtypes} ')
print(f'Raster valid data mask:\t {raster.nodatavals}')
print(f'Raster not valid mask:\t {raster.nodata}')

In [ ]:
# the plot dimensions show the longitude, x, and lattitude, y
show(raster);

### Bands  

From the metadata information block above, "count" shows 1 meaning only 1 band in this raster.  Using 1-based array index, showing the band information.

In [ ]:
# get the band and inspect
band = raster.read(1)

## Descriptions of band
print(f'Band size:\t{band.shape}')
print(f'Band dimensions:\t{band.ndim}')
print(f'Band data type:\t{type(band)}')

In [ ]:
band

In [ ]:
# band has nan values on certain pixels, look at non-nan values
# band is a 2D ndarray, to examine values, it will flatten to 1D
# using to inspect stats only
count = band[~(np.isnan(band))] 

In [ ]:
# stats
print(f'\tmin: {count.min()},\n \tmean: {count.mean()},\n\tmedian: {np.median(count)},\n\tmax: {count.max()}')

In [ ]:
# pink seems to show the light colors better than other colors
fig, ax = plt.subplots()
img = ax.imshow(band[:, :], cmap='pink')
fig.colorbar(img, ax=ax);
ax.set_xlabel('pixel dimensions')
ax.set_ylabel('pixel dimensions')
ax.set_title('Night time mask avg DNB radiance values for ');

### Histogram 

In [ ]:
# this may take several minutes
plt.hist(band)

### Extracting Data from images

In [ ]:
# Open DTM and DSM files
raster2 = rxr.open_rasterio("NigeriaMask2016.tif", masked=True).squeeze()

In [ ]:
## Drop null values
var1 = raster2.to_dataframe('results').dropna(how='all')

In [ ]:
var1 = var1.dropna()

In [ ]:
# Check nodata value for array
raster2.rio.nodata


In [ ]:
raster2

In [ ]:
## Exporting data to a file
var1.to_csv("data_extract.csv")

## Zonal statistics
### Working with shapefiles

In [ ]:
import geopandas as gpd

In [ ]:
###Mask

start_date = '2016-01-01'
end_date =  '2016-12-31'

mySecondMap = geemap.Map()

##VIIRS Nighttime Day/Night Band Composites Version 1

dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(start_date, end_date))
nighttime = dataset.select('avg_rad').median() ##Median is used when masking
nighttimeVis = {min: 0.0, max: 60.0};
mySecondMap.setCenter(center_lat, center_lon, zoomlevel);
mySecondMap.addLayer(nighttime.mask(nighttime), nighttimeVis, '2016 Mask', opacity=0.75)  ##mask function called here.

mySecondMap

In [ ]:
## Quick look at the shape file
gdf = gpd.read_file(r'C:\Users\edidiong.e\Desktop\PERSONAL\OMDENA\Google Earth Engine\nigeria_lga\gadm36_NGA_1.shp')
gdf.plot()

In [ ]:
## conda install pyshp to help view shapefile on map
## conda install -c conda-forge pyshp if Attribute Error


nigeria_shp = r'C:\Users\edidiong.e\Desktop\PERSONAL\OMDENA\Google Earth Engine\nigeria_lga\gadm36_NGA_1.shp'
nigeria = geemap.shp_to_ee(nigeria_shp)
mySecondMap.addLayer(nigeria, {}, 'Nigeria')

##Look at Map
mySecondMap

In [ ]:
out_dir = os.path.expanduser('~/Downloads')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
out_dem_stats = os.path.join(out_dir, 'dem_stats2.csv')  

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(nighttime, nigeria, out_dem_stats, statistics_type='MEAN', scale=1000)